In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
breadbasket = pd.read_csv('/content/bread basket.csv')
breadbasket.head(10)

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend
5,3,Cookies,30-10-2016 10:07,morning,weekend
6,4,Muffin,30-10-2016 10:08,morning,weekend
7,5,Coffee,30-10-2016 10:13,morning,weekend
8,5,Pastry,30-10-2016 10:13,morning,weekend
9,5,Bread,30-10-2016 10:13,morning,weekend


In [ ]:
breadbasket.shape

(20507, 5)

##Data Preparation

In [ ]:
breadbasket['Item'] = breadbasket['Item'].str.strip() #removes spaces from beginning and end
breadbasket.dropna(axis=0, subset=['Transaction'], inplace=True) #removes duplicate invoice
breadbasket['Transaction'] = breadbasket['Transaction'].astype('str') #converting invoice number to be string


In [ ]:
breadbasket.shape

(20507, 5)

In [ ]:
breadbasket['weekday_weekend'].value_counts()

weekday    12807
weekend     7700
Name: weekday_weekend, dtype: int64

#WEEKDAYS

In [ ]:
#separating data for weekdays
myb_weekdays = (breadbasket[breadbasket['weekday_weekend'] =="weekday"]
          .groupby('Transaction')['Item']
          .value_counts().unstack().reset_index().fillna(0)
          .set_index('Transaction'))

In [ ]:
myb_weekdays.head()

Item,Adjustment,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Baguette,Bakewell,Bare Popcorn,Bowl Nic Pitt,Bread,Bread Pudding,Brioche and salami,Brownie,Cake,Cherry me Dried fruit,Chicken Stew,Chimichurri Oil,Chocolates,Christmas common,Coffee,Coffee granules,Coke,Cookies,Crepes,Crisps,Drinking chocolate spoons,Duck egg,Dulce de Leche,Eggs,Ella's Kitchen Pouches,Empanadas,Extra Salami or Feta,Fairy Doors,Farm House,Focaccia,Frittata,Fudge,Gift voucher,Gingerbread syrup,Granola,...,Jammie Dodgers,Juice,Keeping It Local,Kids biscuit,Lemon and coconut,Medialuna,Mighty Protein,Mineral water,Mortimer,Muesli,Muffin,My-5 Fruit Shoot,Nomad bag,Olum & polenta,Panatone,Pastry,Pick and Mix Bowls,Polenta,Raspberry shortbread sandwich,Raw bars,Salad,Sandwich,Scandinavian,Scone,Siblings,Smoothies,Soup,Spanish Brunch,Spread,Tartine,Tea,The BART,The Nomad,Tiffin,Toast,Truffles,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#converting all positive vaues to 1 and everything else to 0
def encoder(x): 
    if(x==0): 
        return False
    if(x>0): 
        return True



## Training Model for Weekdays

In [ ]:
weekdays_sets = myb_weekdays.applymap(encoder)

In [ ]:
#Generatig frequent itemsets
freq_weekdays = apriori(weekdays_sets, min_support=0.02, use_colnames=True)


In [ ]:
#generating rules
weekdays_rules = association_rules(freq_weekdays, metric="lift", min_threshold=1)

In [ ]:

weekdays_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Cake),(Coffee),0.095850,0.485598,0.052238,0.544992,1.122310,0.005693,1.130533
1,(Coffee),(Cake),0.485598,0.095850,0.052238,0.107574,1.122310,0.005693,1.013137
2,(Cake),(Tea),0.095850,0.149390,0.022783,0.237691,1.591080,0.008464,1.115834
3,(Tea),(Cake),0.149390,0.095850,0.022783,0.152505,1.591080,0.008464,1.066850
4,(Coffee),(Cookies),0.485598,0.062002,0.031082,0.064008,1.032361,0.000974,1.002144
5,(Cookies),(Coffee),0.062002,0.485598,0.031082,0.501312,1.032361,0.000974,1.031511
6,(Coffee),(Hot chocolate),0.485598,0.051424,0.025386,0.052279,1.016625,0.000415,1.000902
7,(Hot chocolate),(Coffee),0.051424,0.485598,0.025386,0.493671,1.016625,0.000415,1.015944
8,(Medialuna),(Coffee),0.052726,0.485598,0.028641,0.543210,1.118641,0.003038,1.126123
9,(Coffee),(Medialuna),0.485598,0.052726,0.028641,0.058981,1.118641,0.003038,1.006648


In [ ]:
weekdays_rules.sort_values('confidence', ascending = False, inplace=True)
weekdays_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
15,(Toast),(Coffee),0.037266,0.485598,0.026526,0.711790,1.465802,0.008429,1.784819
10,(Pastry),(Coffee),0.087551,0.485598,0.049634,0.566914,1.167456,0.007119,1.187761
0,(Cake),(Coffee),0.095850,0.485598,0.052238,0.544992,1.122310,0.005693,1.130533
8,(Medialuna),(Coffee),0.052726,0.485598,0.028641,0.543210,1.118641,0.003038,1.126123
5,(Cookies),(Coffee),0.062002,0.485598,0.031082,0.501312,1.032361,0.000974,1.031511
12,(Sandwich),(Coffee),0.075346,0.485598,0.037592,0.498920,1.027434,0.001004,1.026587
7,(Hot chocolate),(Coffee),0.051424,0.485598,0.025386,0.493671,1.016625,0.000415,1.015944
2,(Cake),(Tea),0.095850,0.149390,0.022783,0.237691,1.591080,0.008464,1.115834
3,(Tea),(Cake),0.149390,0.095850,0.022783,0.152505,1.591080,0.008464,1.066850
1,(Coffee),(Cake),0.485598,0.095850,0.052238,0.107574,1.122310,0.005693,1.013137


In [ ]:
weekdays_rules.shape

(16, 9)

##Making Recommendations for Weekdays

In [ ]:
#result=df.loc[df['a'] == 11,'b'].values[0]
#df[df['CLASS']==1]['CONTENT'] 
#df.loc[df['column_name'] == some_value]
# x = weekdays_rules[weekdays_rules['antecedents']=="(Coffee)"]
# x
weekdays_rules['consequents']
#breadbasket[breadbasket['weekday_weekend'] =="weekday"]

15           (Coffee)
10           (Coffee)
0            (Coffee)
8            (Coffee)
5            (Coffee)
12           (Coffee)
7            (Coffee)
2               (Tea)
3              (Cake)
1              (Cake)
11           (Pastry)
13         (Sandwich)
4           (Cookies)
9         (Medialuna)
14            (Toast)
6     (Hot chocolate)
Name: consequents, dtype: object

#WEEKENDS

In [ ]:
myb_weekends = (breadbasket[breadbasket['weekday_weekend'] =="weekend"]
          .groupby('Transaction')['Item']
          .value_counts().unstack().reset_index().fillna(0)
          .set_index('Transaction'))

In [ ]:
myb_weekends.head()

Item,Afternoon with the baker,Alfajores,Argentina Night,Art Tray,Bacon,Baguette,Bakewell,Basket,Bowl Nic Pitt,Bread,Brioche and salami,Brownie,Cake,Caramel bites,Cherry me Dried fruit,Chicken Stew,Chicken sand,Chocolates,Christmas common,Coffee,Coffee granules,Coke,Cookies,Crepes,Crisps,Drinking chocolate spoons,Duck egg,Dulce de Leche,Eggs,Ella's Kitchen Pouches,Empanadas,Extra Salami or Feta,Fairy Doors,Farm House,Focaccia,Frittata,Fudge,Gingerbread syrup,Granola,Half slice Monster,...,Jam,Jammie Dodgers,Juice,Keeping It Local,Kids biscuit,Lemon and coconut,Medialuna,Mighty Protein,Mineral water,Mortimer,Muesli,Muffin,My-5 Fruit Shoot,Nomad bag,Panatone,Pastry,Pick and Mix Bowls,Pintxos,Postcard,Raspberry shortbread sandwich,Salad,Sandwich,Scandinavian,Scone,Smoothies,Soup,Spanish Brunch,Spread,Tacos/Fajita,Tartine,Tea,The Nomad,Tiffin,Toast,Truffles,Tshirt,Valentine's card,Vegan Feast,Vegan mincepie,Victorian Sponge
Transaction,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Training Model for Weekends

In [ ]:
weekends_sets = myb_weekends.applymap(encoder)

In [ ]:
freq_weekends = apriori(weekends_sets, min_support=0.02, use_colnames=True)

In [ ]:
weekends_rules = association_rules(freq_weekends, metric="lift", min_threshold=1)

In [ ]:
weekends_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Alfajores),(Coffee),0.038554,0.465060,0.020181,0.523437,1.125526,0.002251,1.122497
1,(Coffee),(Alfajores),0.465060,0.038554,0.020181,0.043394,1.125526,0.002251,1.005059
2,(Pastry),(Bread),0.083434,0.344578,0.032229,0.386282,1.121027,0.003479,1.067952
3,(Bread),(Pastry),0.344578,0.083434,0.032229,0.093531,1.121027,0.003479,1.011140
4,(Coffee),(Brownie),0.465060,0.051807,0.028614,0.061528,1.187643,0.004521,1.010359
5,(Brownie),(Coffee),0.051807,0.465060,0.028614,0.552326,1.187643,0.004521,1.194930
6,(Cake),(Coffee),0.118675,0.465060,0.059337,0.500000,1.075130,0.004146,1.069880
7,(Coffee),(Cake),0.465060,0.118675,0.059337,0.127591,1.075130,0.004146,1.010220
8,(Cake),(Tea),0.118675,0.130120,0.025602,0.215736,1.657971,0.010160,1.109167
9,(Tea),(Cake),0.130120,0.118675,0.025602,0.196759,1.657971,0.010160,1.097212


In [ ]:
weekends_rules.shape

(26, 9)

##Making Recommendations for Weekends

In [ ]:
myb_weekdays['Sandwich'].sum()

512.0

In [ ]:
pip --version 

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
